In [1]:
config = {}
config['data'] = {"qm9_broad_ir_path":'/home2/kanakala.ganesh/ir_data/qm9_broad_ir.pkl',
                  "vocab_path":'/home2/kanakala.ganesh/CLIP_PART_1/data/qm9_vocab.pkl',
                  "datafiles" : {
                        'train': '/home2/kanakala.ganesh/ir_data/raw_train.pickle',
                        'test':  '/home2/kanakala.ganesh/ir_data/raw_test.pickle',
                        'val':   '/home2/kanakala.ganesh/ir_data/raw_val.pickle'
                        },
                  "normalization" : "unit",
                  "shuffle": True,
                  "batch_size":128,
                  "seq_len":70,
                  "splits":[0.9, 0.1, 0.1],
                  "num_workers":16
                }

In [2]:
from PrepareData import prepare_data
dataloaders, max_charge, num_species = prepare_data(config)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:

for i , data in enumerate(dataloaders['train']):
    data
    for j in data:
        print(j, data[j].shape)
    break

index torch.Size([128])
decoder_inp torch.Size([128, 70])
decoder_tgt torch.Size([128, 70])
IR torch.Size([128, 1801])
tgt_padding_mask torch.Size([128, 70])
num_atoms torch.Size([128])
charges torch.Size([128, 29])
positions torch.Size([128, 29, 3])
one_hot torch.Size([128, 29, 5])
atom_mask torch.Size([128, 29])
edge_mask torch.Size([107648, 1])


In [4]:
import torch


In [5]:
config['molecule_decoder'] = {
    'in_size': 512,
    'latent_size' : 512,
    'hidden_size': 512,
    'n_layers' : 7,
    'n_heads' : 4
}

In [6]:
import pickle 
from models.decoder import LatentToMol

vocab = pickle.load(open(config['data']['vocab_path'], 'rb'))
model = LatentToMol(in_size=config['molecule_decoder']['latent_size'],
                  hidden_size=config['molecule_decoder']['hidden_size'], 
                  n_layers=config['molecule_decoder']['n_layers'], 
                  n_heads = config['molecule_decoder']['n_heads'],
                  seq_len=config['data']['seq_len'], 
                  vocab = vocab)
                  

In [7]:
def set_up_causal_mask(seq_len):
    mask = (torch.triu(torch.ones(seq_len, seq_len)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    mask.requires_grad = False
    return mask

def forward_pass(model, dataloader):
    for i, data in enumerate(dataloader):
        data
        break
    latent = torch.rand(config['data']['batch_size'], config['molecule_decoder']['latent_size'])
    smi = data['decoder_inp']
    tgt = data['decoder_tgt']
    tgt_padding_mask = data['tgt_padding_mask']
    tgt_mask = set_up_causal_mask(config['data']['seq_len'])
    
    pred = model(latent, smi, tgt_mask, tgt_padding_mask)
    
    print(pred.shape)
    return pred

In [8]:
out = forward_pass(model, dataloaders['train'])

torch.Size([128, 70, 25])
